# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations.tsv"
pa14_out_filename = "pa14_core_similarity_associations.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 5)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label
PAO1 id,,,,,
PA1950,PA14_39280,0.353718,1.962739e-157,rbsK,NaN
PA4872,PA14_64440,0.381711,4.085891e-185,NaN,NaN
PA3680,PA14_16770,0.381384,8.943350e-185,NaN,NaN
PA0586,PA14_07650,0.542184,0.000000e+00,NaN,most stable
PA1468,PA14_45450,0.365443,1.080005e-168,NaN,NaN


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5348, 5)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label
PA14 id,,,,,
PA14_09290,PA4224,0.417279,2.157278e-224,pchG,NaN
PA14_63890,PA4832,0.245412,3.443885e-74,NaN,NaN
PA14_53110,PA0863,0.296757,3.538269e-109,NaN,NaN
PA14_38990,PA1975,0.268808,3.474350e-89,NaN,NaN
PA14_48410,PA1227,0.346066,2.595807e-150,NaN,NaN


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA2001, PA4785, PA2011, PA3589, PA2000, PA199..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA1748, PA4785, PA0879, PA2574, PA3925, PA173..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA1748, PA4899, PA3014, PA3331, PA2475, PA182..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA1748, PA4785, PA4236, PA4613, PA4342, PA392..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA2001, PA3633, PA4785, PA4043, PA4557, PA358..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA1950,[KEGG-Pathway-pae00030: Pentose phosphate path...
PA4872,[]
PA3680,[]
PA0586,[]
PA1468,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_39280,[KEGG-Pathway-pae00030: Pentose phosphate path...
PA14_64440,[]
PA14_16770,[]
PA14_07650,[]
PA14_45450,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 6)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,pathways present
PAO1 id,,,,,,
PA1950,PA14_39280,0.353718,1.962739e-157,rbsK,NaN,[KEGG-Pathway-pae00030: Pentose phosphate path...
PA4872,PA14_64440,0.381711,4.085891e-185,NaN,NaN,[]
PA3680,PA14_16770,0.381384,8.943350e-185,NaN,NaN,[]
PA0586,PA14_07650,0.542184,0.000000e+00,NaN,most stable,[]
PA1468,PA14_45450,0.365443,1.080005e-168,NaN,NaN,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5348, 6)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,pathways present
PA14 id,,,,,,
PA14_09290,PA4224,0.417279,2.157278e-224,pchG,NaN,[KEGG-Pathway-pae01053: Biosynthesis of sidero...
PA14_63890,PA4832,0.245412,3.443885e-74,NaN,NaN,[]
PA14_53110,PA0863,0.296757,3.538269e-109,NaN,NaN,[]
PA14_38990,PA1975,0.268808,3.474350e-89,NaN,NaN,[]
PA14_48410,PA1227,0.346066,2.595807e-150,NaN,NaN,[]


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")

**Takeaway:**

Based on the pathways associated with the most and least stable core genes, we find that the most stable core genes tend to be associated with pathways related to cellular maintenance including: protein transport systems, ribosomes, metabolism, type III, IV secretion system which mediates virulence.

There are far fewer KEGG pathways that least stable core genes are found to be associated with. The least stable core genes are mostly associated with different types of metabolism.

A google doc containing the most and least stable core genes and some information about them is [here](https://docs.google.com/spreadsheets/d/1SqEyBvutfbsOTo4afg9GiEzP32ZKplkN1a6MpAQBvZI/edit?usp=sharing).